# Timeseries Sentinel 2 from openEO
In this Notebook, we will extract time series of Sentinel-2 NDVI from a Data Cube for the inundation case

In [16]:
import json

import matplotlib.pyplot as plt
import numpy as np
import openeo
from openeo.rest.job import RESTJob
from openeo.rest.conversions import timeseries_json_to_pandas
import shapely.geometry
import scipy.signal

# Get the 

In [42]:
# Open the GeoJSON file and read the features
with open("../input/test_site_demervallei_WGS84_v2.geojson") as f:
    features = json.load(f)["features"]

print("GeoJSON Features:", features)

# Assuming a single polygon in the GeoJSON file
geometry = shapely.geometry.shape(features[0]["geometry"])  # Extract the geometry of the first feature

# Get the bounds of the polygon (minx, miny, maxx, maxy)
bounds = geometry.bounds

print("Polygon Geometry:", geometry)
print("Bounds:", bounds)

GeoJSON Features: [{'type': 'Feature', 'properties': {'id': 1}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[4.968710486638257, 51.00869219207143], [4.980305670916424, 51.00784563198833], [4.979613659415014, 51.00357796037619], [4.973508463494545, 51.00378616074288], [4.968512914917515, 51.00519269175937], [4.968710486638257, 51.00869219207143]]]]}}]
Polygon Geometry: MULTIPOLYGON (((4.968710486638257 51.00869219207143, 4.980305670916424 51.00784563198833, 4.979613659415014 51.00357796037619, 4.973508463494545 51.00378616074288, 4.968512914917515 51.00519269175937, 4.968710486638257 51.00869219207143)))
Bounds: (4.968512914917515, 51.00357796037619, 4.980305670916424, 51.00869219207143)


In [43]:
start_date = "2024-05-01"
end_date = "2024-05-08"
bands = [ "B08", "B11", "SCL"]

In [44]:
con  = openeo.connect("https://openeo.vito.be").authenticate_oidc(provider_id="egi")
bbox = {"west": geometry.bounds[0] , "south":geometry.bounds[1] , "east": geometry.bounds[2], "north": geometry.bounds[3], "crs": "EPSG:4326"}
dates = (start_date, end_date)

datacube = con.load_collection("SENTINEL2_L2A", temporal_extent=dates, bands=bands, spatial_extent = bbox)

# Extract band11
band11 = datacube.band("B11")

# Calculate NDMI
ndmi = datacube.ndvi(nir="B08", red="B11")

# Extract the Scene Classification (SCL) layer
classification = datacube.band("SCL")



Authenticated using refresh token.


## Masking with a kernel

In [45]:
# Create a cloud mask using SCL values (4 and 5 are clouds and cloud shadows)
cloud_mask = ~((classification == 4) | (classification == 5))

# Apply a Gaussian kernel to smooth the mask
g = scipy.signal.windows.gaussian(11, std=1.5)  # 11x11 kernel with std=1.5
kernel = np.outer(g, g)
kernel = kernel / kernel.sum()  # Normalize kernel
cloud_mask = cloud_mask.apply_kernel(kernel)

# Threshold the smoothed mask
cloud_mask = cloud_mask > 0.1  # Threshold to make it binary again


In [48]:
# Apply the mask to both Band 11 and NDMI
masked_band11 = band11.mask(mask)
masked_ndmi = ndmi.mask(mask)

## Download the data

In [39]:
# Save all time slices of Band 11 as a multiband GeoTIFF
job = band11.save_result(
    format="GTiff",
    options={"tiled": True, "bands_dimension": "t"}  # Combine all time steps into bands
)

# Execute the batch job
job = job.execute_batch()

# Download the single multiband GeoTIFF file
job_results = job.get_results()
result_files = job_results.download_files(target="../output/band11")

print("Multiband GeoTIFF file saved to:", result_files)

0:00:00 Job 'j-250116085722487d8de39bfb2f566f1b': send 'start'
0:00:15 Job 'j-250116085722487d8de39bfb2f566f1b': created (progress 0%)
0:00:20 Job 'j-250116085722487d8de39bfb2f566f1b': queued (progress 0%)
0:00:27 Job 'j-250116085722487d8de39bfb2f566f1b': queued (progress 0%)
0:00:35 Job 'j-250116085722487d8de39bfb2f566f1b': queued (progress 0%)
0:00:44 Job 'j-250116085722487d8de39bfb2f566f1b': queued (progress 0%)
0:00:57 Job 'j-250116085722487d8de39bfb2f566f1b': running (progress N/A)
0:01:12 Job 'j-250116085722487d8de39bfb2f566f1b': running (progress N/A)
0:01:31 Job 'j-250116085722487d8de39bfb2f566f1b': running (progress N/A)
0:01:55 Job 'j-250116085722487d8de39bfb2f566f1b': running (progress N/A)
0:02:25 Job 'j-250116085722487d8de39bfb2f566f1b': finished (progress 100%)
Multiband GeoTIFF file saved to: [WindowsPath('../output/band11/openEO_2024-01-02Z.tif'), WindowsPath('../output/band11/openEO_2024-01-05Z.tif'), WindowsPath('../output/band11/openEO_2024-01-07Z.tif'), WindowsPath(

In [ ]:
# Save all time slices of Band 11 as a multiband GeoTIFF
job = masked_ndmi.save_result(
    format="GTiff",
    options={"tiled": True, "bands_dimension": "t"}  # Combine all time steps into bands
)

# Execute the batch job
job = job.execute_batch()

# Download the single multiband GeoTIFF file
job_results = job.get_results()
result_files = job_results.download_files(target="ndmi")

print("Multiband GeoTIFF file saved to:", result_files)

In [49]:
# Save the SCL band as a GeoTIFF file
job = classification.save_result(format="GTiff", options={"tiled": True})

# Execute the batch job to process the data and generate the output
job = job.execute_batch()

# Download the resulting GeoTIFF file
job_results = job.get_results()
result_files = job_results.download_files(target="../output/scl_output")

print("SCL GeoTIFF saved to:", result_files)

0:00:00 Job 'j-2501160936344e539d163f22853eb6c9': send 'start'
0:02:13 Job 'j-2501160936344e539d163f22853eb6c9': created (progress 0%)
0:02:18 Job 'j-2501160936344e539d163f22853eb6c9': queued (progress 0%)
0:02:24 Job 'j-2501160936344e539d163f22853eb6c9': queued (progress 0%)
0:02:32 Job 'j-2501160936344e539d163f22853eb6c9': queued (progress 0%)
0:02:42 Job 'j-2501160936344e539d163f22853eb6c9': queued (progress 0%)
0:02:55 Job 'j-2501160936344e539d163f22853eb6c9': finished (progress 100%)
SCL GeoTIFF saved to: [WindowsPath('../output/scl_output/openEO_2024-05-01Z.tif'), WindowsPath('../output/scl_output/openEO_2024-05-04Z.tif'), WindowsPath('../output/scl_output/openEO_2024-05-06Z.tif'), WindowsPath('../output/scl_output/job-results.json')]
